## 1. Data Preparation

In [ ]:
import pandas as pd

## Departure Delay Dataset
Dataset from Kaggle: **"Flight Status Prediction"** by *Rob Mulla*  
Source: https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022/data

## Import CSV file into a DataFrame

In [ ]:
flights_original = pd.read_csv('data/Flights_2022_7.csv', low_memory = False) # Importing July dataset

In [ ]:
flights_original = pd.DataFrame(flights_original[[
                    'Month',
                    'DayofMonth',
                    'DayOfWeek',
                    'FlightDate',
                    'Operating_Airline ',
                    'Origin',
                    'OriginState',
                    'OriginAirportID',
                    'Dest',
                    'DestState',
                    'DestAirportID',
                    'CRSDepTime',
                    'DepDelay',
                    'DepDelayMinutes',
                    'DepDel15',
                    'TaxiOut',
                    'Distance',
                    'DistanceGroup'
                ]])

## Weather Events Dataset
Dataset from Kaggle: **"US Weather Events (2016 - 2022)"** by *Sobhan Moosavi*  
Source: https://www.kaggle.com/datasets/sobhanmoosavi/us-weather-events

Our group decided to consider the factor of weather events causing a departure delay to flights

In [ ]:
weather = pd.read_csv("data/WeatherEvents_Jan2016-Dec2022.csv")

In [ ]:
from datetime import datetime
from dateutil import parser

In [ ]:
# Extracting July 2022 data
# Example date: "2016-01-07 00:34:00"
def extract_month_year(row):
    # return datetime.strptime(row["StartTime(UTC)"], '%y-%m-%d %H:%M:%S').year
    parsed = parser.parse(row["StartTime(UTC)"])
    return str(parsed.month) +  "-" + str(parsed.year)


weather['Month-Year'] = weather.apply(extract_month_year, axis=1)
weather_2021 = weather[weather['Month-Year'] == "7-2022"]

In [ ]:
# Prune for only severe or other severity (All "Other" severity indicates hail, and all hail has "Other" severity)
severe_weather = weather_2021.query("Severity == 'Severe' or Severity == 'Other'")

In [ ]:
iata_icao = pd.read_csv("data/iata-icao.csv")
iata_to_icao_map = iata_icao.set_index('iata')['icao'].to_dict()

flights_original["SevereWeatherEvent"] = "N"


In [ ]:
icao_to_event_times = {}

for index in severe_weather.index:
    icao = severe_weather["AirportCode"][index]
    start = severe_weather["StartTime(UTC)"][index]
    end = severe_weather["EndTime(UTC)"][index]

    if icao in icao_to_event_times:
        icao_to_event_times[icao].append([start, end])
    else:
        icao_to_event_times[icao] = [[start, end]]

In [ ]:
def check_event(row):
    iata = row["Origin"]

    if iata not in iata_to_icao_map:
        return "N"
    
    icao = iata_to_icao_map[iata]
    if icao not in icao_to_event_times:
        return "N"
    
    flight_date = str(row["FlightDate"])
    flight_time = str(row["CRSDepTime"]) # Format: "hhmm" or "hmm" or "mm" or "m"
    if len(flight_time) < 4:
        flight_time = "0" + flight_time
    if len(flight_time) < 4:
        flight_time = "0000"

    date_time_string = flight_date + " " + flight_time[0:2] + ":" + flight_time[2:4] + ":00"
    flight_date_time = parser.parse(date_time_string)

    events = icao_to_event_times[icao]
    for event in events:
        if flight_date_time >= parser.parse(event[0]) and flight_date_time <= parser.parse(event[1]):
            return "Y"
        
    return "N"
    

flights_original['SevereWeatherEvent'] = flights_original.apply(check_event, axis=1)
flights_original.to_csv('data/Flights_Prepared.csv')